In [23]:
bucket_file = "s3://ms-syntheamass-1m-mldata/syntheamassCSV/patient_info2/observations.csv"

data = spark.read.format("csv").option("inferSchema", True).option("header", False).load(bucket_file)
data.createOrReplaceTempView("data")
data = spark.sql("""
select _c0 as patient,
_c1 as height, _c2 as weight, _c3 as BMI, _c4 as cholesterol,
_c5 as sodium, _c6 as glucose
from data""")

data.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+------+-----+-----------+------+-------+
|             patient|height|weight|  BMI|cholesterol|sodium|glucose|
+--------------------+------+------+-----+-----------+------+-------+
|00c343e4-e4eb-4c6...|176.61|116.12|37.23|      163.0| 141.0|   83.0|
|00eec485-f89d-472...| 172.3| 94.04|31.68|      207.0| 140.0|  116.0|
|017299f2-de8f-43a...|177.38|135.75|43.15|      161.0| 139.0|   72.0|
|018dd53e-4b90-41c...|163.33|136.74|51.26|      162.0| 143.0|   71.0|
|0190ce14-8897-495...| 161.1| 72.92| 28.1|      196.0| 137.0|   75.0|
|01e7a88d-34ac-494...|173.35|105.97|35.27|      169.0| 143.0|   85.0|
|01f649ca-232e-4c1...|151.11|102.05|44.69|      199.0| 140.0|   94.0|
|0216c768-a4c0-46a...|169.24| 96.32|33.63|      197.0| 140.0|   64.0|
|023230b8-01ca-4d3...|157.84| 92.49|37.13|      179.0| 142.0|   88.0|
|02357156-a39f-45f...|185.33|109.37|31.84|      173.0| 143.0|   74.0|
|0259aa6f-d531-43f...|163.33|100.15|37.54|      233.0| 140.0|  100.0|
|029f2ed6-60a0-4d4..

In [24]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="patient", outputCol="ID")
indexed = indexer.fit(data).transform(data)

indexed = indexed.drop("patient")
indexed.show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+------+-----+-----------+------+-------+-------+
|height|weight|  BMI|cholesterol|sodium|glucose|     ID|
+------+------+-----+-----------+------+-------+-------+
|176.61|116.12|37.23|      163.0| 141.0|   83.0|21687.0|
| 172.3| 94.04|31.68|      207.0| 140.0|  116.0|12249.0|
|177.38|135.75|43.15|      161.0| 139.0|   72.0|28222.0|
|163.33|136.74|51.26|      162.0| 143.0|   71.0| 5018.0|
| 161.1| 72.92| 28.1|      196.0| 137.0|   75.0|23116.0|
|173.35|105.97|35.27|      169.0| 143.0|   85.0|10539.0|
|151.11|102.05|44.69|      199.0| 140.0|   94.0|13891.0|
|169.24| 96.32|33.63|      197.0| 140.0|   64.0| 4396.0|
|157.84| 92.49|37.13|      179.0| 142.0|   88.0|24275.0|
|185.33|109.37|31.84|      173.0| 143.0|   74.0|21453.0|
|163.33|100.15|37.54|      233.0| 140.0|  100.0|17018.0|
|170.46|111.64|38.42|      171.0| 144.0|   90.0|29378.0|
|170.45|112.29|38.65|      199.0| 144.0|   78.0| 9794.0|
|166.01|  92.5|33.56|      178.0| 143.0|   72.0|30135.0|
|175.46| 92.34|29.99|      160.

In [25]:
# creating a vector of features for each row
from pyspark.ml.feature import VectorAssembler

features = ('ID', 'height', 'weight', 'BMI', 'cholesterol', 'sodium', 'glucose')

assembler = VectorAssembler(inputCols=features, outputCol = "features")

raw_data = assembler.transform(indexed)
raw_data.select("features").show(truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------------------------------+
|features                                       |
+-----------------------------------------------+
|[21687.0,176.61,116.12,37.23,163.0,141.0,83.0] |
|[12249.0,172.3,94.04,31.68,207.0,140.0,116.0]  |
|[28222.0,177.38,135.75,43.15,161.0,139.0,72.0] |
|[5018.0,163.33,136.74,51.26,162.0,143.0,71.0]  |
|[23116.0,161.1,72.92,28.1,196.0,137.0,75.0]    |
|[10539.0,173.35,105.97,35.27,169.0,143.0,85.0] |
|[13891.0,151.11,102.05,44.69,199.0,140.0,94.0] |
|[4396.0,169.24,96.32,33.63,197.0,140.0,64.0]   |
|[24275.0,157.84,92.49,37.13,179.0,142.0,88.0]  |
|[21453.0,185.33,109.37,31.84,173.0,143.0,74.0] |
|[17018.0,163.33,100.15,37.54,233.0,140.0,100.0]|
|[29378.0,170.46,111.64,38.42,171.0,144.0,90.0] |
|[9794.0,170.45,112.29,38.65,199.0,144.0,78.0]  |
|[30135.0,166.01,92.5,33.56,178.0,143.0,72.0]   |
|[29460.0,175.46,92.34,29.99,160.0,141.0,98.0]  |
|[15720.0,173.81,108.56,35.93,172.0,139.0,71.0] |
|[23009.0,162.92,102.96,38.79,200.0,143.0,77.0] |


In [30]:
# scaling the feature vectors and creating a column with the output
from pyspark.ml.feature import StandardScaler

standardscaler = StandardScaler().setInputCol("features").setOutputCol("scaled_features")

raw_data = standardscaler.fit(raw_data).transform(raw_data)
raw_data.select("scaled_features").show(5, truncate = True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|     scaled_features|
+--------------------+
|[2.47080205020966...|
|[1.39552977880842...|
|[3.21533524512460...|
|[0.57170123520782...|
|[2.63360816123238...|
+--------------------+
only showing top 5 rows

In [11]:
# training the k-means model using the scaled features column
from pyspark.ml.clustering import KMeans

kmeans = KMeans(featuresCol = "scaled_features").setK(5).setSeed(1)
model = kmeans.fit(raw_data)



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# performing the k-means clustering job
predictions = model.transform(raw_data)

In [33]:
# listing the number of data points in each cluster
clusters = predictions.select("prediction")
clusters.groupby("prediction").agg({'prediction': 'count'}).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----------------+
|prediction|count(prediction)|
+----------+-----------------+
|         1|             8296|
|         3|             2600|
|         4|             5203|
|         2|             6372|
|         0|             7934|
+----------+-----------------+

In [34]:
# performing a heirarchical clustering job

from pyspark.ml.clustering import BisectingKMeans

# training the model
bkm = BisectingKMeans(featuresCol="scaled_features").setK(5).setSeed(1)
model = bkm.fit(raw_data)

# creating predictions 
predictions_b = model.transform(raw_data)

# listing the number of points in each cluster
cm = predictions_b.select("prediction")
cm.groupby("prediction").agg({'prediction': 'count'}).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----------------+
|prediction|count(prediction)|
+----------+-----------------+
|         1|             8430|
|         3|             5189|
|         4|             3010|
|         2|             4951|
|         0|             8825|
+----------+-----------------+

In [35]:
from pyspark.ml.evaluation import ClusteringEvaluator

# Make predictions


#Evaluate using Silhouette score
evaluator = ClusteringEvaluator()

silhouette_k = evaluator.evaluate(predictions)
silhouette_h = evaluator.evaluate(predictions_b)
print(f"""K-means silhouette with squared euclidean distance = {str(silhouette_k)} \n 
    Hierarchical silhouette with squared euclidean distance = {str(silhouette_h)}""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

K-means silhouette with squared euclidean distance = -0.131598264118536 
 
        Hierarchical silhouette with squared euclidean distance = -0.23110782587064593

In [1]:
# performing a similar cluster job with more variables including patient demographics

bucket_file = "s3://ms-syntheamass-1m-mldata/syntheamassCSV/patient_info2/patient_info_and_observations2.csv"

data2 = spark.read.format("csv").option("inferSchema", True).option("header", False).load(bucket_file)
data2.createOrReplaceTempView("data2")
data2 = spark.sql("""
select _c0 as patient,
_c1 as height, _c2 as weight, _c3 as BMI, _c4 as cholesterol,
_c5 as sodium, _c6 as birth_date, _c7 as marital, _c8 as race, 
_c9 as ethnicity
from data2""")

# data2.show()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1596028402588_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark.ml.feature import StringIndexer
# inputs = ["patient", "marital", "ethnicity", "race"]
# outputs = ["ID", "indexed_marital", "indexed_ethnicity", "indexed_race"]

# drop rows with any missing values:
data2 = data2.dropna()

indexer = StringIndexer(inputCol="patient", outputCol="ID")
indexed = indexer.fit(data2).transform(data2)

indexer = StringIndexer(inputCol="marital", outputCol="indexed_marital")
indexed = indexer.fit(indexed).transform(indexed)

indexer = StringIndexer(inputCol="ethnicity", outputCol="indexed_ethnicity")
indexed = indexer.fit(indexed).transform(indexed)

indexer = StringIndexer(inputCol="race", outputCol="indexed_race")
indexed = indexer.fit(indexed).transform(indexed)

indexed.createOrReplaceTempView("indexed")

indexed = spark.sql("""
select ID, height, weight, BMI, sodium, birth_date, indexed_marital, indexed_ethnicity, indexed_race
from indexed
""")

# indexed = indexed.drop("patient")
# indexed = indexed.drop("marital")
# indexed = indexed.drop("ethnicity")
# indexed = indexed.drop("race")

indexed.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------+------+-----+------+-------------------+---------------+-----------------+------------+
|     ID|height|weight|  BMI|sodium|         birth_date|indexed_marital|indexed_ethnicity|indexed_race|
+-------+------+------+-----+------+-------------------+---------------+-----------------+------------+
|12200.0| 172.3| 94.04|31.68| 140.0|1959-06-24 00:00:00|            0.0|              3.0|         1.0|
|28107.0|177.38|135.75|43.15| 139.0|1963-12-03 00:00:00|            1.0|              1.0|         0.0|
| 4998.0|163.33|136.74|51.26| 143.0|1955-11-19 00:00:00|            0.0|              0.0|         0.0|
|23024.0| 161.1| 72.92| 28.1| 137.0|1939-03-17 00:00:00|            0.0|              0.0|         0.0|
|10498.0|173.35|105.97|35.27| 143.0|1958-03-09 00:00:00|            1.0|              6.0|         3.0|
|13838.0|151.11|102.05|44.69| 140.0|1965-10-17 00:00:00|            0.0|              9.0|         0.0|
| 4376.0|169.24| 96.32|33.63| 140.0|1955-07-14 00:00:00|        

In [3]:
from datetime import datetime
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

# import pyspark.sql.functions as F
# import pyspark.sql.types as T
# from pyspark.sql.functions import lit, when, col, regexp_extract, datediff

today = datetime.today()

convert_to_age = udf(lambda birth_date: today.year - birth_date.year, IntegerType())

indexed = indexed.withColumn("age", convert_to_age(indexed.birth_date))

indexed = indexed.drop("birth_date")
   
# convert_to_age("1974-05-03 00:00:00")

# birth_dates = indexed.select("birth_date").rdd.map(lambda birth_date: convert_to_age(birth_date))

# birth_dates.take(5)

# the with column needs a column as its second argument
# this can be return through a udf 


    


# my_udf = F.UserDefinedFunction(convert_to_age, T.DateType())

# indexed = indexed.withColumn('age', my_udf('birth_date'))

# indexed.withColumn('test', datediff(str(today), lit(indexed.birth_date))/365).show()




# type(indexed.select("birth_date").rdd.map(lambda birth_date: today.year - birth_date[0].year))

# indexed.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
from pyspark.ml.feature import VectorAssembler

features = [feature for feature in indexed.columns if feature != "ID"]
# features = ('height', 'weight', 'BMI', 'cholesterol', 'sodium', 'glucose')

assembler = VectorAssembler(inputCols=features, outputCol = "features")

raw_data = assembler.transform(indexed)
raw_data.select("features").show(truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------------------------+
|features                                     |
+---------------------------------------------+
|[172.3,94.04,31.68,140.0,0.0,3.0,1.0,61.0]   |
|[177.38,135.75,43.15,139.0,1.0,1.0,0.0,57.0] |
|[163.33,136.74,51.26,143.0,0.0,0.0,0.0,65.0] |
|[161.1,72.92,28.1,137.0,0.0,0.0,0.0,81.0]    |
|[173.35,105.97,35.27,143.0,1.0,6.0,3.0,62.0] |
|[151.11,102.05,44.69,140.0,0.0,9.0,0.0,55.0] |
|[169.24,96.32,33.63,140.0,0.0,0.0,0.0,65.0]  |
|[157.84,92.49,37.13,142.0,1.0,0.0,0.0,65.0]  |
|[185.33,109.37,31.84,143.0,0.0,18.0,0.0,58.0]|
|[163.33,100.15,37.54,140.0,1.0,0.0,0.0,50.0] |
|[170.46,111.64,38.42,144.0,1.0,0.0,0.0,72.0] |
|[170.45,112.29,38.65,144.0,0.0,0.0,0.0,62.0] |
|[166.01,92.5,33.56,143.0,1.0,14.0,0.0,59.0]  |
|[175.46,92.34,29.99,141.0,0.0,5.0,0.0,68.0]  |
|[173.81,108.56,35.93,139.0,1.0,12.0,2.0,71.0]|
|[162.92,102.96,38.79,143.0,0.0,1.0,0.0,66.0] |
|[166.72,76.58,27.55,139.0,0.0,0.0,0.0,38.0]  |
|[181.81,122.25,36.98,142.0,0.0,18.0,0.0

In [5]:
# scaling the feature vectors and creating a column with the output
from pyspark.ml.feature import StandardScaler

standardscaler = StandardScaler().setInputCol("features").setOutputCol("scaled_features")

raw_data = standardscaler.fit(raw_data).transform(raw_data)
raw_data.select("scaled_features").show(1, truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------------------------------------------------------------------------------------------------------------------+
|scaled_features                                                                                                                      |
+-------------------------------------------------------------------------------------------------------------------------------------+
|[18.81097714552532,5.449289792499681,5.045078238264507,54.406909224147476,0.0,0.5734279758748507,1.105977502549777,4.316379804851266]|
+-------------------------------------------------------------------------------------------------------------------------------------+
only showing top 1 row

In [6]:
# training the k-means model using the scaled features column
from pyspark.ml.clustering import KMeans

kmeans = KMeans(featuresCol = "scaled_features").setK(5).setSeed(1)
model = kmeans.fit(raw_data)

# performing the k-means clustering job
predictions = model.transform(raw_data)

# listing the number of data points in each cluster
clusters = predictions.select("prediction")
clusters.groupby("prediction").agg({'prediction': 'count'}).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----------------+
|prediction|count(prediction)|
+----------+-----------------+
|         1|             8291|
|         3|             4841|
|         4|             7682|
|         2|             4518|
|         0|             4952|
+----------+-----------------+

In [7]:
# performing a heirarchical clustering job

from pyspark.ml.clustering import BisectingKMeans

# training the model
bkm = BisectingKMeans(featuresCol="scaled_features").setK(5).setSeed(1)
model = bkm.fit(raw_data)

# creating predictions 
predictions_b = model.transform(raw_data)

# listing the number of points in each cluster
cm = predictions_b.select("prediction")
cm.groupby("prediction").agg({'prediction': 'count'}).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----------------+
|prediction|count(prediction)|
+----------+-----------------+
|         1|             8013|
|         3|             3320|
|         4|             3649|
|         2|             7595|
|         0|             7707|
+----------+-----------------+

In [8]:
from pyspark.ml.evaluation import ClusteringEvaluator

#Evaluate using Silhouette score
evaluator = ClusteringEvaluator()

silhouette_k = evaluator.evaluate(predictions)
silhouette_h = evaluator.evaluate(predictions_b)
print(f"""K-means silhouette with squared euclidean distance = {str(silhouette_k)} \n 
Hierarchical silhouette with squared euclidean distance = {str(silhouette_h)}""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

K-means silhouette with squared euclidean distance = 0.14375978381517615 
 
Hierarchical silhouette with squared euclidean distance = 0.09577782660795764

In [8]:
from pyspark.mllib.linalg.distributed import RowMatrix

mat = RowMatrix(predictions.scaled_features, numRows=-1, numCols=-1)

# Compute the top 2 principal components.
# Principal components are stored in a local dense matrix.
pc = mat.computePrincipalComponents(2)

# Project the rows to the linear space spanned by the top 2 principal components.
# projected = mat.multiply(pc)
pc.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rows should be an RDD of vectors, got <class 'pyspark.sql.column.Column'>
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/mllib/linalg/distributed.py", line 101, in __init__
    raise TypeError("rows should be an RDD of vectors, got %s" % type(rows))
TypeError: rows should be an RDD of vectors, got <class 'pyspark.sql.column.Column'>



In [11]:
predictions.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- height: double (nullable = true)
 |-- weight: double (nullable = true)
 |-- BMI: double (nullable = true)
 |-- cholesterol: double (nullable = true)
 |-- sodium: double (nullable = true)
 |-- ID: double (nullable = false)
 |-- indxed_marital: double (nullable = false)
 |-- indexed_ethnicity: double (nullable = false)
 |-- indexed_race: double (nullable = false)
 |-- age: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- scaled_features: vector (nullable = true)
 |-- prediction: integer (nullable = false)

In [12]:
predictions_sample = predictions.sample(False, 0.1, seed=42)
predictions_sample.createOrReplaceTempView("predictions_sample")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
%%sql -q -o sqlResults
SELECT scaled_features, prediction
FROM predictions_sample

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
%%local
sqlResults.head()

Output()